In [2]:
from collections import Counter

import datasets
import tiktoken
import numpy as np

/home/hmosousa/Projects/tradutor_dataset/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Raw

In [3]:
raw = datasets.load_dataset("liaad/PTradutor", "raw")

#### Count the number of tokens

In [3]:
encoder = tiktoken.get_encoding("cl100k_base")

On the train set.

In [4]:
n_tokens_train_en = sum(len(encoder.encode(example["en"])) for example in raw["train"])
print(f"Number of tokens in the training set in english: {n_tokens_train_en}")

Number of tokens in the training set in english: 307348193


In [5]:
n_tokens_train_pt = sum(len(encoder.encode(example["pt"])) for example in raw["train"])
print(f"Number of tokens in the training set in portuguese: {n_tokens_train_pt}")

Number of tokens in the training set in portuguese: 404133911


In [6]:
print(f"Number of tokens in the training set in total: {n_tokens_train_en + n_tokens_train_pt}")

Number of tokens in the training set in total: 711482104


On the test set.

In [7]:
n_tokens_test_en = sum(len(encoder.encode(example["en"])) for example in raw["test"])
print(f"Number of tokens in the test set in english: {n_tokens_test_en}")

Number of tokens in the test set in english: 74899


In [8]:
n_tokens_test_pt = sum(len(encoder.encode(example["pt"])) for example in raw["test"])
print(f"Number of tokens in the test set in portuguese: {n_tokens_test_pt}")

Number of tokens in the test set in portuguese: 100911


## Number of tokens that start with the same `n_chars`

In [9]:
n_chars = 60
raw = raw.map(lambda x: {"start": x["pt"][:n_chars], "end": x["pt"][-n_chars:]}, num_proc=96)


Map (num_proc=96): 100%|██████████| 857/857 [00:00<00:00, 1857.27 examples/s]


In [10]:
raw["start"]

KeyError: 'start'

In [5]:
_ , unique_starts_idxs = np.unique(raw["start"], return_index=True, axis=0)
print(f"Number of unique starts {len(unique_starts_idxs)} out of {len(raw)}")

Number of unique starts 3862445 out of 3944464


In [6]:
_ , unique_ends_idxs = np.unique(raw_train["end"], return_index=True, axis=0)
print(f"Number of unique ends {len(unique_ends_idxs)} out of {len(raw_train)}")

Number of unique ends 3870791 out of 3944464


In [7]:
unique_idxs = set.intersection(set(unique_starts_idxs), set(unique_ends_idxs))
print(f"Number of unique starts and ends {len(unique_idxs)} out of {len(raw_train)}")


Number of unique starts and ends 3806658 out of 3944464


In [8]:
filtered_raw = raw_train.select(list(unique_idxs))
filtered_raw

Dataset({
    features: ['idx', 'source', 'domain', 'pt', 'en', 'start', 'end'],
    num_rows: 3806658
})

# Clean

In [3]:
clean = datasets.load_dataset("liaad/PTradutor", "clean")

Looking at the most common starts.

In [31]:
start = clean["train"].map(lambda x: {"start": x["pt"][:100]}, num_proc=96)
count = Counter(start["start"])
count.most_common(30)

[('É este o nosso plano de festas para Black Friday! Fica atento, participa e habilita-te a ganhar prém',
  873),
 ('As restrições derivadas do combate à Pandemia são uma Ditadura Sanitária. Estamos todos reféns. O Pr',
  444),
 ('Acabei de publicar uma foto em Portugal', 361),
 ('Sugiro que as pessoas Ignorem as Diretrizes do Governo e ajam de acordo com o bom Senso. -O Gabinete',
  299),
 ('Acabei de publicar uma foto em Lisbon, Portugal', 203),
 ('Acabei de publicar um vídeo em Portugal', 189),
 ('A tua odd da sorte é no Toni Martinez a marcar ao Sporting! Se isto acontecer habilitas-te a ganhar ',
  184),
 ('A tua odd da sorte é no Sporting vence e acima de 1,5 golos! Se isto acontecer habilitas-te a ganhar',
  181),
 ('A tua odd da sorte é no Sporting ou empate e acima de 1,5 golos! Se isto acontecer habilitas-te a ga',
  178),
 ('A tua odd da sorte é no Nuno Santos a marcar ao Porto! Se isto acontecer habilitas-te a ganhar 10€ e',
  178),
 ('A tua odd da sorte é no Sporting vence

Looking at the most common ends.

In [36]:
end = clean["train"].map(lambda x: {"end": x["pt"][-100:]}, num_proc=96)
count = Counter(end["end"])
count.most_common(30)

Map (num_proc=96): 100%|██████████| 3263624/3263624 [00:15<00:00, 206785.40 examples/s]


[('o plano de festas para Black Friday! Fica atento, participa e habilita-te a ganhar prémios à grande!',
  873),
 ('Acabei de publicar uma foto em Portugal', 361),
 ('Parlamento. Sugiro que as pessoas Ignorem as Diretrizes do Governo e ajam de acordo com o bom Senso.',
  352),
 (' ou empate e acima de 1,5 golos! Se isto acontecer habilitas-te a ganhar 10€ em Bónus! Achas que dá?',
  343),
 ('Acabei de publicar uma foto em Lisbon, Portugal', 203),
 ('Acabei de publicar um vídeo em Portugal', 189),
 ('itária entregues às autoridades locais. -O Governo Perdeu a Confiança das Pessoas e já não a merece.',
  189),
 ('i Martinez a marcar ao Sporting! Se isto acontecer habilitas-te a ganhar 10€ em Bónus! Achas que dá?',
  184),
 ('ting vence e acima de 1,5 golos! Se isto acontecer habilitas-te a ganhar 10€ em Bónus! Achas que dá?',
  181),
 ('o Nuno Santos a marcar ao Porto! Se isto acontecer habilitas-te a ganhar 10€ em Bónus! Achas que dá?',
  178),
 ('ting vence ao Intervalo e Final! Se is

Filter out duplicates

In [41]:
initial_list = raw["train"].filter(lambda example: (example['idx'], example['pt'][:n_chars]), num_proc=96)

Filter (num_proc=96): 100%|██████████| 3263624/3263624 [00:01<00:00, 2058857.20 examples/s]


In [42]:
initial_list

Dataset({
    features: ['idx', 'source', 'domain', 'pt', 'en'],
    num_rows: 3263624
})